In [2]:
import numpy as np, pandas as pd, json, glob, os, openai
# from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
from tqdm.auto import tqdm
# import shutil
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct


In [3]:
client = QdrantClient(path="../data/qdrant_local")  # 데이터는 ./qdrant_local 폴더에 저장됨

# 2️⃣ 컬렉션 이름 및 파라미터 정의
collection_name = "medical_qa"


In [4]:
# client.delete_collection(collection_name=collection_name)

In [5]:
# client.create_collection(
#         collection_name=collection_name,
#         vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
#     )
model = SentenceTransformer("BAAI/bge-m3")


In [6]:
path = '../../../data/1.데이터/Training/02.라벨링데이터/'
path = '../../../medical_data/02.라벨링데이터/'
path = '/home/ethan/git_rp/data/1.데이터/Training/02.라벨링데이터/'
# os.listdir(path)
# for name_ in os.listdir(path):
#     print(name_)
#     collection_name = name_

# NOTE Create Collection
# client.recreate_collection(
# collection_name=collection_name,
# vectors_config=VectorParams(size=1024, distance=Distance.COSINE))
# for i in tqdm(os.listdir(path)):
#     for j in tqdm(os.listdir(path+'/'+i)):
#         with open(path+i+'/'+j, "r", encoding="utf-8-sig") as f:
#             data = json.load(f)
        
        
#         embedding = model.encode(data["question"]).tolist()
        
#         client.upsert(
#         collection_name=collection_name,
#         points=[
#             PointStruct(
#                 id=data["qa_id"],
#                 vector=embedding,
#                 payload={
#                     "domain": data["domain"],
#                     "q_type": data["q_type"],
#                     "question": data["question"],
#                     "answer": data["answer"],
#                     },
#                 )
#             ],
#         )
#     print(f"{i} done===============")
    
        
        
        
        
        
    #     break
    # break        

# RAG Description 추가

## Qdrant Collection 추가

In [25]:
content_collection = 'Major_Content'
collection_name
client.recreate_collection(
collection_name=content_collection,
vectors_config=VectorParams(size=1024, distance=Distance.COSINE))




/tmp/ipykernel_1874709/552978515.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

### Collection Upload

In [29]:
data

{'c_id': '53715_1',
 'domain': 2,
 'source': 7,
 'source_spec': '세브란스병원',
 'creation_year': '2024',
 'content': '소아 신경외과에서 시행하는 선택적 후신경근 절제술은 약물이나 재활치료로 효과가 거의 없는 경직성 사지마비 환자에게 주로 시행됩니다. 이 수술은 척수 후각으로 들어오는 감각신경 중 과민한 신호를 선택적으로 차단하여 경직을 완화하는 데 목적이 있습니다. 특히 미숙아로 태어난 아이들 중 2세에서 6세 사이의 환자에게 적합하며, 경직으로 인해 걸을 수 있는 상태일 때 가장 좋은 결과를 기대할 수 있습니다. 이 수술은 심한 경직으로 인해 발생할 수 있는 근골격 변형, 척추측만증, 고관절 탈구 등을 줄이는 데 효과적입니다. 수술 과정은 다음과 같습니다. 환자를 엎드린 자세로 눕히고 하지에 근전도 검사를 위한 장치를 부착한 후, 요추부 후궁을 절제하여 경막을 노출시킵니다. 경막을 열고 신경을 확인한 뒤, 후신경근을 분리합니다. 이후 전기 자극술을 이용해 정상 및 비정상적인 반응을 보이는 신경세근을 선별하여 절제합니다. 구체적으로, 후신경근을 구분하고 근전도로 각 신경을 자극하여 반응 강도를 확인한 뒤, 기준보다 강한 반응을 보이는 신경을 선택적으로 절제합니다. 수술 후 경막을 복원하고 피부를 봉합하며, 수술 시간은 약 4~5시간이 소요됩니다. 수술 후 발생할 수 있는 합병증으로는 신경통증(다리 저림, 감각 이상, 통증 등), 방광 조절 이상, 장운동 이상 등이 있습니다. 이러한 증상은 대부분 일시적이며, 환자마다 차이가 있습니다. 수술 후 약 1주일 후부터 물리치료 및 운동치료를 시작합니다. 수술의 효과를 극대화하고 유지하기 위해 지속적인 재활치료가 필요하며, 경우에 따라 보조기나 정형외과적 치료가 병행될 수 있습니다. 수술 중에는 척수 신경을 노출하고 자극을 주기 때문에, 이에 따른 증상이 수술 후 나타날 수 있습니다. 또한, 수술 중 환자의 상태에 따라 긴급히 수술 방법

In [ ]:
model.encode(data['content'].split('. ')).shape

(23, 1024)

In [ ]:
folder_path = '../../../data/1.데이터/Training/01.원천데이터/'
# 폴더 구분
points = []
for folder_ in tqdm(os.listdir(folder_path)):
    print(folder_path+folder_)
    # 파일 선택
    print(f"Folder count : {len(os.listdir(folder_path+folder_))}")
    for file_ in tqdm(os.listdir(folder_path+folder_)):
        # print(folder_path+folder_+"/"+file_)
        with open(folder_path+folder_+"/"+file_, "r", encoding="utf-8-sig") as f:
            data = json.load(f)
        
        embed = model.encode(data['content'].split('. ')).shape
        payload = {
            **data,  # 편, 장, 절, 조, content
        }

        point = PointStruct(
            id=file_.split('.')[0],
            vector=embed,
            payload=payload
        )
        points.append(point)    
        
    
        # break
    # break
    

In [ ]:

vectors_config={
        "text_embedding": VectorParams(size=1024, distance=Distance.COSINE),
        "content_embedding": VectorParams(size=1024, distance=Distance.COSINE),
    }

{'c_id': '53715_1',
 'domain': 2,
 'source': 7,
 'source_spec': '세브란스병원',
 'creation_year': '2024',
 'content': '소아 신경외과에서 시행하는 선택적 후신경근 절제술은 약물이나 재활치료로 효과가 거의 없는 경직성 사지마비 환자에게 주로 시행됩니다. 이 수술은 척수 후각으로 들어오는 감각신경 중 과민한 신호를 선택적으로 차단하여 경직을 완화하는 데 목적이 있습니다. 특히 미숙아로 태어난 아이들 중 2세에서 6세 사이의 환자에게 적합하며, 경직으로 인해 걸을 수 있는 상태일 때 가장 좋은 결과를 기대할 수 있습니다. 이 수술은 심한 경직으로 인해 발생할 수 있는 근골격 변형, 척추측만증, 고관절 탈구 등을 줄이는 데 효과적입니다. 수술 과정은 다음과 같습니다. 환자를 엎드린 자세로 눕히고 하지에 근전도 검사를 위한 장치를 부착한 후, 요추부 후궁을 절제하여 경막을 노출시킵니다. 경막을 열고 신경을 확인한 뒤, 후신경근을 분리합니다. 이후 전기 자극술을 이용해 정상 및 비정상적인 반응을 보이는 신경세근을 선별하여 절제합니다. 구체적으로, 후신경근을 구분하고 근전도로 각 신경을 자극하여 반응 강도를 확인한 뒤, 기준보다 강한 반응을 보이는 신경을 선택적으로 절제합니다. 수술 후 경막을 복원하고 피부를 봉합하며, 수술 시간은 약 4~5시간이 소요됩니다. 수술 후 발생할 수 있는 합병증으로는 신경통증(다리 저림, 감각 이상, 통증 등), 방광 조절 이상, 장운동 이상 등이 있습니다. 이러한 증상은 대부분 일시적이며, 환자마다 차이가 있습니다. 수술 후 약 1주일 후부터 물리치료 및 운동치료를 시작합니다. 수술의 효과를 극대화하고 유지하기 위해 지속적인 재활치료가 필요하며, 경우에 따라 보조기나 정형외과적 치료가 병행될 수 있습니다. 수술 중에는 척수 신경을 노출하고 자극을 주기 때문에, 이에 따른 증상이 수술 후 나타날 수 있습니다. 또한, 수술 중 환자의 상태에 따라 긴급히 수술 방법

# Question

In [7]:
query_data = {
    "qa_id": 4286,
    "domain": 17,
    "q_type": 1,
    "question": "65세 남자가 심질환으로 현재 aspirin을 복용 중이다. 이 환자에서 aspirin 복용으로 인해 나타날 수 있는 소견으로 가장 적절한 것은?  \n1) PTT 증가  \n2) TT 증가  \n3) 혈소판 감소  \n4) PT 증가  \n5) BT 증가",
    "answer": "5) BT 증가"
}

# domain, q_type 기준으로 필터 설정
filters = models.Filter(
    must=[
        models.FieldCondition(
            key="domain",
            match=models.MatchValue(value=query_data["domain"])
        ),
        models.FieldCondition(
            key="q_type",
            match=models.MatchValue(value=query_data["q_type"])
        ),
    ]
)

In [8]:
query_vector = model.encode(query_data["question"]).tolist()

In [9]:
results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3,
    query_filter=filters
)

/tmp/ipykernel_1874709/1981553409.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [10]:
for i, r in enumerate(results):
    print(f"{i+1}. Score: {r.score:.4f}")
    print(f"Question: {r.payload['question']}")
    print(f"Answer: {r.payload['answer']}\n")


1. Score: 0.7091
Question: 65세 남성이 급성 심근경색으로 응급실에 내원하였다. 이 환자의 초기 치료로 권장되는 aspirin의 최소 부하 용량은 얼마인가?  
1) 50 mg  
2) 75 mg  
3) 100 mg  
4) 160~325 mg  
5) 500 mg
Answer: 4) 160~325 mg

2. Score: 0.7024
Question: 65세 남성이 관상동맥 스텐트 시술 후 항혈전 치료를 받고 있다. 티클로피딘과 아스피린을 병용 투여할 경우, 다음 중 주요한 치료 효과로 기대되는 것은 무엇인가?  
1) 총 사망률 감소  
2) 혈전성 혈소판 감소성 자반증(TTP) 발생 감소  
3) 비치명적 심근경색 발생 감소  
4) 백혈구 감소증 발생 감소  
5) 경미한 출혈 증가
Answer: 3) 비치명적 심근경색 발생 감소

3. Score: 0.6967
Question: 60세 남성이 갑작스러운 심한 가슴 통증으로 응급실에 내원하였습니다. 심전도에서 ST 분절 상승이 관찰되었고, 혈액 검사에서 트로포닌 수치가 상승하였습니다. 이 환자에서 가장 우선적으로 시행해야 할 치료는 무엇인가요?

1) 심장 초음파  
2) 혈전 용해제 투여  
3) 응급 관상동맥 중재술(PCI) 
4) 니트로글리세린 설하정  
5) 아스피린 복용
Answer: 3) 응급 관상동맥 중재술(PCI)



In [12]:
# # 3️⃣ 임베딩 모델 로드 (bge-m3)
# model = SentenceTransformer("BAAI/bge-m3")

# # 5️⃣ question 임베딩 생성
# # embedding = model.encode(data["question"]).tolist()

# # 6️⃣ Qdrant에 데이터 업로드
# client.upsert(
#     collection_name=collection_name,
#     points=[
#         PointStruct(
#             id=data["qa_id"],
#             vector=embedding,
#             payload={
#                 "domain": data["domain"],
#                 "q_type": data["q_type"],
#                 "question": data["question"],
#                 "answer": data["answer"],
#             },
#         )
#     ],
# )

# print("✅ 데이터 업로드 완료!")

In [13]:
# if collection_name not in [c.name for c in client.get_collections().collections]:
#     client.create_collection(
#         collection_name=collection_name,
#         vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
#     )
#     print(f"✅ 컬렉션 '{collection_name}' 생성 완료")
# else:
#     print(f"ℹ️ 컬렉션 '{collection_name}' 이미 존재")